In [1]:
import pandas as pd
import numpy as np
import csv
import json
import collections as cl

In [2]:
TARGET_DATE = '0915'

In [4]:
label_df = pd.read_csv('../data/label' + TARGET_DATE + '.csv')
time_df = pd.read_csv('../data/2018' + TARGET_DATE + 'time.csv')
label_id = pd.read_csv('../../label_id.csv')

In [5]:
label_id

,label,work
0,1,その他
1,2,書類作成
2,3,シリンダー準備
3,4,原紙
4,5,インクローラー調整
5,6,巻取準備（上）
6,7,巻取準備（下）
7,8,サンプル作成
8,9,インク調整
9,10,シリンダー片付け


In [8]:
label_numpy = label_df.values

In [6]:
label_numpy 

array([[72732, 13, '人がいない'],
       [74403, 2, '書類作成'],
       [74416, 11, '作業台での作業'],
       ...,
       [165520, 2, '書類作成'],
       [165753, 1, 'その他'],
       [165910, 13, '人がいない']], dtype=object)

In [9]:
def get_time (day):
    if len(day) == 5:
        return int(day[0:1]), int(day[1:3]) , int(day[3:5])
    else :
        return  int(day[0:2]), int(day[2:4]), int(day[4:6])

In [10]:
for index , row in enumerate(label_numpy):
    print(row)
    print(str(label_numpy[index + 1][0]))
    break

[73522 13 '人がいない']
74003


## 正解データの作成

In [12]:
f = open('../data/' + TARGET_DATE + 'formated_label.csv', 'w')
writer = csv.writer(f) 
writer.writerow(('time', 'label'))
# 15fpsなので4fpsに直す
for index, row in enumerate(label_numpy):
    try:
        hour ,  minute, second = get_time(str(row[0]))
        end_hour , end_minute ,end_second = get_time(str(label_numpy[index + 1][0]))
        while hour <= end_hour and( minute*60 + second)  <= (end_minute* 60 + end_second):
            id_num = label_id [label_id['work'].isin([row[2]])]['label']
            writer.writerow( (str(hour)+':' + str(minute) + ':' +str(second) , int(row[1])))
            second += 1
            if second == 60 :
                minute +=1 
                second = 0
                if minute == 60:
                    hour +=1
                    minute = 0
    #最後のindexの処理
    except:
        hour ,  minute, second = get_time(str(row[0]))
        writer.writerow( (str(hour)+':' + str(minute) + ':' +str(second) , int(row[1])))
f.close()

## 教師データの作成

In [1]:
def create_train_data (position , time, label_path , fps):
    
    formated_label_df = pd.read_csv(label_path)
    data_dict = cl.OrderedDict()
    image_path_list =[] # [[4fps分の画像のpath], [4fps分の画像のpath]]
    # 4fpsにしてる
    for index, row in formated_label_df.iterrows():
        target = 0
        hour , minute ,second = row['time'].split(':')
        target_time = time[(time.hour == int(hour)) & (time.minute == int(minute)) & (time.second == int(second))]
        if(len(target_time.index) != 0):
            time_list = []
            for i in range(fps):
                time_list.appned(target_time.iloc[target + int(len(target_time.index) /fps) *i ]['path'])
        image_path_list.append(time_list)
                
    for row in image_path_list:
        for path in row:
            
        posi = position[row['path']]
        path_list = row['path'].split('/')
        key = path_list[0].replace('2018/09') + path_list[1]
        data_dict[key]
    return pd.merge(position, new_time, on='path')

In [2]:
pf = open('data/' +  TARGET_DATE + 'central_coordinate.json', 'r')
position_json= json.load(f) #JSON形式で読み込む
formated_data = create_train_data(position_json, time_df , './data/' + TARGET_DATE + 'formated_label.csv' , 4)

NameError: name 'time_df' is not defined

In [11]:
formated_data.to_csv('./data/' + TARGET_DATE + 'formated_data.csv' , index = False)

## otherラベルの作成

In [32]:
label_df = pd.read_csv('data/formated_label.csv',header = None)

In [33]:
label_df.head()

,0,1,2
0,7:55:55,0,朝処理
1,7:55:56,0,朝処理
2,7:55:57,0,朝処理
3,7:55:58,0,朝処理
4,7:55:59,0,朝処理


In [35]:
hour =7
minute = 55
second = 55
while hour <= 10:
    while minute < 60:
        while second < 60:
            time = str(hour) + ":" + str(minute) + ":" + str(second)
            if (label_df[label_df[0] == time ].shape[0] ==0):
                print(time)
                s = pd.Series([time, 68, 'その他'])
                label_df = label_df.append(s ,ignore_index=True)
            second += 1
        second = 0
        minute += 1
    minute =0
    hour += 1
    
while minute < 24:
    while second < 60:
        time = str(hour) + ":" + str(minute) + ":" + str(second)
        if (label_df[label_df[0] == time ].shape[0] ==0):
            print(time)
            s = pd.Series([time, 68, 'その他'])
            label_df = label_df.append(s ,ignore_index=True)
        second += 1
    second = 0
    minute += 1
        

7:59:45
7:59:46
7:59:47
7:59:48
7:59:49
7:59:50
7:59:51
7:59:52
7:59:53
7:59:54
7:59:55
7:59:56
7:59:57
7:59:58
7:59:59
8:0:0
8:0:1
8:0:2
8:0:3
8:0:4
8:0:5
8:0:6
8:0:7
8:0:8
8:0:9
8:0:10
8:0:11
8:0:12
8:0:13
8:0:14
8:0:15
8:0:16
8:0:17
8:0:18
8:0:19
8:0:20
8:0:21
8:0:22
8:0:23
8:0:24
8:0:25
8:0:26
8:0:27
8:0:28
8:0:29
8:0:30
8:0:31
8:0:32
8:0:33
8:0:34
8:0:35
8:0:36
8:0:37
8:0:38
8:0:39
8:0:40
8:0:41
8:0:42
8:0:43
8:0:44
8:0:45
8:0:46
8:0:47
8:0:48
8:0:49
8:0:50
8:0:51
8:0:52
8:0:53
8:0:54
8:0:55
8:0:56
8:0:57
8:0:58
8:0:59
8:1:0
8:1:1
8:1:2
8:1:3
8:1:4
8:1:5
8:1:6
8:1:7
8:1:8
8:1:9
8:1:10
8:1:11
8:1:12
8:1:13
8:1:14
8:1:31
8:1:32
8:1:33
8:1:34
8:1:35
8:1:36
8:1:37
8:1:38
8:1:39
8:1:40
8:1:41
8:1:42
8:1:43
8:1:44
8:1:45
8:1:46
8:1:47
8:1:48
8:1:49
8:1:50
8:1:51
8:1:52
8:1:53
8:1:54
8:1:55
8:1:56
8:1:57
8:1:58
8:1:59
8:2:0
8:2:1
8:2:2
8:2:3
8:2:4
8:2:5
8:2:6
8:2:7
8:2:8
8:2:9
8:2:10
8:2:11
8:2:12
8:2:13
8:2:14
8:2:15
8:2:16
8:2:17
8:2:18
8:2:19
8:2:20
8:2:21
8:2:22
8:2:23
8:2:24
8:2:25


In [37]:
label_df.tail()

,0,1,2
12709,11:5:0,68,その他
12710,11:5:1,68,その他
12711,11:5:2,68,その他
12712,11:5:3,68,その他
12713,11:5:4,68,その他


In [39]:
label_df.to_csv('./data/formated_label_other.csv' , index = False)

In [ ]:
predict_df = pd.read_csv('data/formated_label_other.csv',header=None)

## CRF用

In [90]:
label_df = pd.read_csv('data/formated_label_other.csv')
label_id = pd.read_csv('data/label_id.csv')
time_df = pd.read_csv('data/20180903time.csv')
position_df = pd.read_csv('data/central_coordinate.csv' , header=None)

In [65]:
for index, row in position_df.iterrows():
    row[0] =row[0].strip('./kataoka/all_frames/')

position_df = position_df.rename(columns={0: 'path' , 1 :'arm',2: 'head',3:'body',4:'leg'})
formated_data = pd.merge(position_df, time_df, on='path')

In [101]:
formated_data['label'] = 0
for index, row in label_df.iterrows():
    times = row[0].split(':')
    label = label_id[label_id['id'] == row[1]]['Class']
    index = label.index[0]
    extraction = formated_data[(formated_data['hour'] == int(times[0])) & (formated_data['minute'] == int(times[1])) & (formated_data['second'] == int(times[2]))]
    for ex_index in extraction.index:
        formated_data.iloc[ex_index,8] = str(label[index])        

In [99]:
extraction.head()

,arm,head,body,leg,hour,minute,second,label
356322,"[331, 474]","[0, 0]","[0, 0]","[0, 0]",7,55,55,製造開始準備
356627,"[333, 474]","[0, 0]","[0, 0]","[0, 0]",7,55,55,製造開始準備
356958,"[333, 474]","[0, 0]","[0, 0]","[0, 0]",7,55,55,製造開始準備
357179,"[333, 474]","[0, 0]","[0, 0]","[0, 0]",7,55,55,製造開始準備
357527,"[333, 474]","[0, 0]","[0, 0]","[0, 0]",7,55,55,製造開始準備


In [102]:
extraction = formated_data[formated_data['label'] != 0]
extraction = extraction.drop('path', axis = 1)

In [103]:
extraction.to_csv('data/0903_data_crf.csv' , index=None)

In [106]:
label_id

,Class,id
0,製造開始準備,0
1,作業確認,1
2,会話,2
3,装置起動,3
4,清掃,4
5,端末操作,5
6,書類作成,6
7,版袋運搬,7
8,マグネットシリンダー運搬,8
9,仕様確認,9
